<a href="https://colab.research.google.com/github/Krankile/npmf/blob/main/notebooks/training_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Kernel setup

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%%capture
!pip install wandb more_itertools
!git clone https://github.com/Krankile/npmf.git

In [20]:
# https://wandb.ai/authorize
!wandb login

wandb: Currently logged in as: ankile (krankile). Use `wandb login --relogin` to force relogin


## General setup

In [21]:
%%capture
!cd npmf && git pull

import math
import multiprocessing
import os
import pickle
import random
from collections import Counter, defaultdict
from dataclasses import asdict, dataclass
from datetime import datetime, timedelta
from operator import itemgetter
from typing import Callable, List, Tuple
from functools import partial
from glob import glob
from enum import Enum
from pathlib import Path

from more_itertools import chunked

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

from npmf.utils import Problem
from npmf.utils.colors import main, main2, main3
from npmf.utils.dataset import EraDataset, EraController
from npmf.utils.dtypes import fundamental_types
from npmf.utils.eikon import column_mapping
from npmf.utils.tests.utils import pickle_df
from npmf.utils.wandb import get_datasets, put_dataset, put_nn_model, get_processed_data
from npmf.utils.training import EarlyStop, to_device, TqdmPostFix, loss_fns, get_naive_pred
from npmf.utils.models import models

from numpy.ma.core import outerproduct
from pandas.tseries.offsets import BDay, Day
from sklearn.preprocessing import MinMaxScaler, minmax_scale
from torch import nn
from torch.utils.data import DataLoader, Dataset, ConcatDataset

import wandb as wb

In [22]:
np.seterr(all="raise")

mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[main, main2, main3, "black"])
mpl.rcParams['figure.figsize'] = (6, 4)  # (6, 4) is default and used in the paper

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [24]:
!nvidia-smi

Tue Jun  7 19:33:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    27W /  70W |   1542MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [25]:
pre_proc_data_dir = None
np.random.seed(69)


# Get some data

In [26]:
%%capture
reload_data = not True

if reload_data or not "stock_df" in vars():
    names = ["stock-data:final", "fundamental-data:final", "meta-data:final", "macro-data:final"]

    stock_df, fundamental_df, meta_df, macro_df = get_datasets(names=names, project="master")

    stock_df = stock_df.drop(columns=["close_price", "currency"]).astype({"market_cap": np.float32})
    fundamental_df = fundamental_df.drop(columns="period_end_date").astype(fundamental_types)
    macro_df.iloc[:, 1:] = macro_df.iloc[:, 1:].astype(np.float32)

# Create the loop! (Like Hans Gude Gudesen)

In [27]:
# Check if it's necessary to calculate naive loss every epoch
def get_epoch_loss(model, optimizer, dataloader, loss_fn, device, run_type, conf) -> Tuple[np.array, np.array, np.array]:
    model_losses = []
    naive_losses = []
    y_preds = []
    for data, meta_cont, meta_cat, target in to_device(dataloader, device):

        optimizer.zero_grad()
        y_pred: torch.Tensor = model(data, meta_cont, meta_cat)

        naive_pred = get_naive_pred(data, target, device, conf)
        naive_loss = loss_fn(target.clone(), naive_pred)
        loss = loss_fn(target, y_pred)

        model_losses.append(loss.item())
        naive_losses.append(naive_loss.item())
        y_preds.append(y_pred.detach().cpu().numpy())

        if run_type == "train":
            loss.backward()
            optimizer.step()

    return model_losses, naive_losses, np.concatenate(y_preds, axis=0)

In [28]:
def eras_ahead_loss(model, data_loaders, optimizer, conf):
    model_infront = []
    naive_infront = []

    with torch.no_grad():
        for loader in data_loaders:
            model_loss, naive_loss, _ = get_epoch_loss(model, optimizer, loader, loss_fns["mape_2"], device, "inference", conf)
        
            model_infront += model_loss
            naive_infront += naive_loss
    
    return np.array(model_infront), np.array(naive_infront)

In [29]:
def train_one_era(run, model, optimizer, data_train, data_val, stopper, losses, device, conf, pbar):

    for epoch in range(conf.max_epochs):
        epoch_losses = dict(train=[], val=[])
        
        pbar.update_postfix({"epoch": epoch})
        for run_type, dataloader in {"train": data_train, "val": data_val}.items():
            model.train(run_type == "train")
            
            epoch_model_loss, naive_losses, y_preds = get_epoch_loss(model, optimizer, dataloader, loss_fns[conf[f"{run_type}_loss"]], device, run_type, conf)
            epoch_losses[run_type] += epoch_model_loss

            epoch_loss = np.mean(epoch_losses[run_type])
            losses[run_type].append(epoch_loss)

            run.log({f"epoch_{run_type}": epoch_loss, "epoch": epoch, "ticker_var": y_preds.std(axis=0).mean(), "self_var": y_preds.std(axis=1).mean()})

        pbar.update_postfix({"train_loss": np.mean(epoch_losses["train"]), "val_loss": np.mean(epoch_losses["val"]), "naive": np.mean(naive_losses)})


        # TODO: Implement checkpointing of the best model according to val_loss
        if run_type == "val" and stopper(epoch_losses["val"]):
            losses["epoch_lens"].append(epoch + 1)
            break

    return epoch_losses["train"], epoch_losses["val"]

In [35]:
def train(config, project=None, entity=None, enablewb=True) -> nn.Module:
    
    mode = "online" if enablewb else "offline"
    with wb.init(config=config, project=project, entity=entity, job_type="training", mode=mode) as run:

        conf = run.config
        print(conf)

        pre_proc_data_dir = None
        if conf.use_pre_proc_data:
            pre_proc_data_dir = get_processed_data(run, kind=conf.forecast_w)

        run.config.update(dict(pre_proc_data_dir=pre_proc_data_dir))
        conf = run.config
        
        model = models[conf.model](**conf).to(device)

        # Try decreasing learning rate underway
        optimizer = torch.optim.Adam(model.parameters(), lr=conf.learning_rate)

        losses = dict(train=[], val=[], epoch_lens=[])

        eras = EraController(start_date=conf.start_date, end_metric_start_date=conf.end_date, queue_length=conf.queue_length, stock_df=stock_df, fundamental_df=fundamental_df, meta_df=meta_df, macro_df=macro_df, conf=conf)
        pbar = TqdmPostFix(eras, total=eras.total)
        stopper = EarlyStop(conf.patience, conf.min_delta, model=(model if conf.checkpoint else None), pbar=pbar)

        for i, (data_train, data_val) in enumerate(pbar):
            
            pbar.set_description(f"Era {eras.date} [{i+1}/{eras.total}]")

            train_losses, val_losses = train_one_era(
                run=run, 
                model=model, 
                optimizer=optimizer, 
                data_train=data_train, 
                data_val=data_val,
                stopper=stopper.reset(),
                losses=losses,
                device=device, 
                conf=conf,
                pbar=pbar,
            )

            loaders_infront, loaders_end = eras.validation_loaders()
            model_infront, naive_infront = eras_ahead_loss(model, loaders_infront, optimizer, conf)
            model_end, naive_end = eras_ahead_loss(model, loaders_end, optimizer, conf)

            metric_loss = 0.5*(np.mean(model_infront/(naive_infront+1e-6)-1) +  np.mean(model_end/(naive_end+1e-6)-1))

            run.log({"era_train": np.mean(train_losses), "era_val" : np.mean(val_losses),"model_infront": np.mean(model_infront),
                     "naive_infront": np.mean(naive_infront), "model_end": np.mean(model_end), "naive_end": np.mean(naive_end),
                     "metric_loss": metric_loss, **eras.loader_to_na_dict[eras.date], "time": eras.date.timestamp(), "era": i})
            
            pbar.update_postfix(dict(metric_loss=metric_loss))

        if conf.save_model:
            put_nn_model(model, run)

    return model, losses

In [31]:
def get_params_from_data(stock_df, fundamental_df, meta_df, macro_df, params):
    meta_cont_len = 1
    meta_cat_len = np.array([len(meta_df[col].unique()) for col in meta_df.iloc[:,1:] if col != "founding_year"]) + 1
    
    stock_feats = 1
    macro_feats = (macro_df.shape[1]-1)
    funda_feats = (fundamental_df.loc[:,"revenue":].shape[1] - 1) + 2

    n_features = stock_feats + macro_feats + funda_feats

    if params.get("feature_subset") is not None:
        n_features = len(params["feature_subset"])
    
    data_given_params = dict(
        meta_cont_lens=(meta_cont_len, 1),
        meta_cat_lens=list(map(lambda x: (x, int(math.ceil(x**0.25))), meta_cat_len)),
        out_len=1 if params["forecast_problem"] == Problem.volatility.name else params["forecast_w"] if params["forecast_problem"] == Problem.market_cap.name else funda_feats,
        input_size=n_features,
    )

    return data_given_params

# Run the loop! (Like Odd-Geir Lademo)

In [32]:
def mape_loss_2(target: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor:
    mask = (~target.isnan()) & target.isfinite()
    denom = mask.sum(dim=1)
    target[target != target] = 0
    l = (
        (((y_pred - target).abs() / (target.abs() + 1e-6) * mask)).sum(dim=1) / denom
    ).mean()
    return l

def mse_loss_2(target: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor:

    mask = ~target.isnan() & target.isfinite()
    denom = mask.sum(dim=1)

    target[target != target] = 0
    l = ((((y_pred - target) ** 2 * mask)).sum(dim=1) / denom).mean()
    return l

loss_fns = dict(
    mape_2=mape_loss_2,
    mse_2=mse_loss_2,
)

In [36]:
forecast_problem = Problem.fundamentals

params_human = dict(
    forecast_problem=forecast_problem.name,

    cpus=1,
    training_w=240,
    forecast_w=20,
    start_date="2000-12-31",
    end_date="2018-5-31",
    save_model=True,
    batch_size=512,
    use_pre_proc_data=False,
    clamp=2.5,
    dtype="float32",
    queue_length=2,

    include_past=True,
    checkpoint=True,
    feature_subset=None,

    fundamental_targets=None,
)

params_wb = dict(
    max_epochs=100,
    patience=10,
    min_delta=0.0001,
    learning_rate=0.0001,

    hd=64,
    dropout=0.1,
    num_layers=6,
    channels=64,
    kernel_size=5,

    meta_hd=16,

    model="TcnV2",
    activation="relu",

    train_loss=forecast_problem.loss.mape,
    val_loss=forecast_problem.loss.mape,
)

params_from_data = get_params_from_data(stock_df, fundamental_df, meta_df, macro_df, {**params_human, **params_wb})

config = {  
    **params_human,
    **params_wb,
    **params_from_data,
}

In [37]:
enablewb = True
sweepid = None  # "krankile/master/q8hau0w8"

if sweepid:
    count = 500 # number of runs to execute
    wb.agent(sweepid, partial(train,config=config, enablewb=enablewb), count=count)

else:
    model, losses = train(config=config, project="master-test", entity="krankile", enablewb=enablewb)

{'forecast_problem': 'fundamentals', 'cpus': 1, 'training_w': 240, 'forecast_w': 20, 'start_date': '2014-12-31', 'end_date': '2018-5-31', 'save_model': True, 'batch_size': 512, 'use_pre_proc_data': False, 'clamp': 2.5, 'dtype': 'float32', 'queue_length': 2, 'include_past': True, 'checkpoint': False, 'feature_subset': None, 'fundamental_targets': None, 'max_epochs': 100, 'patience': 10, 'min_delta': 0.0001, 'learning_rate': 0.0001, 'hd': 64, 'dropout': 0.1, 'num_layers': 6, 'channels': 64, 'kernel_size': 5, 'meta_hd': 16, 'model': 'TcnV2', 'activation': 'relu', 'train_loss': 'mape_2', 'val_loss': 'mape_2', 'meta_cont_lens': [1, 1], 'meta_cat_lens': [[110, 4], [6, 2], [91, 4], [285, 5], [3, 2], [5, 2], [7, 2], [14, 2], [58, 3]], 'out_len': 18, 'input_size': 37}


Era 2016-10-31 00:00:00 [23/42]:  55%|█████▍    | 23/42 [15:30<12:48, 40.47s/it, epoch=10, train_loss=1.55e+3, val_loss=2.66e+4, naive=0.662, triggers=10/10, best_loss=1.2e+4, metric_loss=3.64e+4]


epoch,▂▂▁▁▂▃▄▅▆▇█▂▁▁▂▃▂▂▂▃▂▁▂▃▁▂▂▃▂▁▁▁▂▂▁▂▁▂▂▂
epoch_train,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▄▂▁▂▄▂▁▁▂▁▁▁▃▁▂▁▃▄▁
epoch_val,▁▁▁█▇▇▇▆▆▅▅▁▁▂▂▂▁▁▁▁▂▁▁▁▂▂▁▁▃▂▁▂▂▁▂▁▃▂▂▄
era,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
era_train,▂▂▁▁▁▁▁▁▁▁▃█▁▁▁▁▁█▂▁▇▅▁
era_val,▁▁█▁▁▃▁▂▁▁▂▄▁▄▄▁▄▃▂▆▄▂▅
fundamental,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
macro,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metric_loss,▁▁▁▁▁▁▁▁▁▁▅▆▁▁▁▃▁▃▆▁█▆▁
model_end,█▇▄▃▂▂▁▁▂▂▄▆▂▂▂▂▂▁▁▃▄▂▃
model_infront,▁▆▆▁▂▃▁▂▁▄▄▄▄▅▂▂▆▂▂█▄▃▂


KeyboardInterrupt: ignored